In [45]:
import torch
import torch.nn as nn # 인공신경망 모델의 재료들을 담고 있음
import torch.optim as optim # 최적화를 위해
import torch.nn.functional as F # nn 모듈의 함수 버전
from torchvision import transforms, datasets

In [46]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)

cuda


In [47]:
EPOCHS = 30
BATCH_SIZE = 64

In [48]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [49]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    download = True, 
    transform = transform
)

testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transform
)

In [60]:
train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)
test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

In [74]:
## 모델의 가중치 변수들이 들어가는 연선을 선언
## nn.Linear 클래스는 선형결합을 수행하는 객체를 만듦
## 픽셀값을 입력받아 가중치를 행렬곱하고 편향을 더해 출력
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256) #(픽셀값, 편향)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10) #출력값 10개 -> 10개 중 값이 가장 큰 클래스가 이 모델의 예측값.
        
    # 자세한 흐름은 forward() 함수에 정의     
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [82]:
model = Net().to(DEVICE) # 모델의 파라미터를 DEVICE로

In [83]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [84]:
# 학습에 들어가는 모든 연산 (학습할 model, 데이터를 공급, 최적화)
def train(model, train_loader, optimizer):
    model.train() # 습관 들이기
    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad() # 반복때마다 기울기를 새로 계산
        # 위 호출 후 학습 데이터에 대한 모델의 예측값인 output 받아옴
        output = model(data)
        # output과 레이블인 target의 오차를 구해야 함!
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        

In [85]:
#
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    # 평가 가정에서는 기울기를 구하지 않는다.
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # item() : 1개의 값을 가진 텐서 값을 가져오는 함수
            
            # 가장 큰 값을 가진 클래스가 모델의 예측
            # 예측과 정답을 비교하여 일치할 경우 correct에 1을 더함
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy


In [86]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print('[{}] Test loss: {:.4f}, Accuracy: {:.2f}%'.format(
        epoch, test_loss, test_accuracy))

[1] Test loss: 0.8476, Accuracy: 68.54%
[2] Test loss: 0.6719, Accuracy: 76.15%
[3] Test loss: 0.5890, Accuracy: 78.98%
[4] Test loss: 0.5390, Accuracy: 80.65%
[5] Test loss: 0.5092, Accuracy: 82.20%
[6] Test loss: 0.5054, Accuracy: 81.85%
[7] Test loss: 0.5080, Accuracy: 81.75%
[8] Test loss: 0.4928, Accuracy: 82.71%
[9] Test loss: 0.4664, Accuracy: 83.27%
[10] Test loss: 0.4732, Accuracy: 82.79%
[11] Test loss: 0.4451, Accuracy: 84.10%
[12] Test loss: 0.4496, Accuracy: 83.67%
[13] Test loss: 0.4549, Accuracy: 84.16%
[14] Test loss: 0.4251, Accuracy: 85.18%
[15] Test loss: 0.4252, Accuracy: 85.01%
[16] Test loss: 0.4270, Accuracy: 84.98%
[17] Test loss: 0.4131, Accuracy: 85.50%
[18] Test loss: 0.4289, Accuracy: 84.65%
[19] Test loss: 0.4137, Accuracy: 85.38%
[20] Test loss: 0.4016, Accuracy: 85.83%
[21] Test loss: 0.4193, Accuracy: 85.37%
[22] Test loss: 0.3908, Accuracy: 86.22%
[23] Test loss: 0.3922, Accuracy: 86.17%
[24] Test loss: 0.3944, Accuracy: 86.00%
[25] Test loss: 0.3838, A